In [2]:
import requests

POE_GQL_URL = "https://poe.com/api/gql_POST"
POE_GQL_MODEL_HASH = "b24b2f2f6da147b3345eec1a433ed17b6e1332df97dea47622868f41078a40cc"

class PoeError(Exception):
    pass

class PoeClient:
    def __init__(self, bot_name, access_key):
        self.bot_name = bot_name
        self.access_key = access_key
        self.client = requests.Session()
        self.client.headers.update({
            "Authorization": f"Bearer {self.access_key}",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
            "Origin": "https://poe.com",
            "Referer": "https://poe.com",
            "Sec-Fetch-Dest": "empty",
            "Sec-Fetch-Mode": "cors",
            "Sec-Fetch-Site": "same-origin",
            "poegraphql": "1"
        })

    def get_model_list(self, language_code=None):
        payload = {
            "queryName": "ExploreBotsListPaginationQuery",
            "variables": {
                "categoryName": "defaultCategory",
                "count": 150
            },
            "extensions": {
                "hash": POE_GQL_MODEL_HASH
            }
        }

        if language_code:
            self.client.headers.update({"Cookie": f"Poe-Language-Code={language_code}; p-b=1"})

        response = self.client.post(POE_GQL_URL, json=payload)

        if not response.ok:
            raise PoeError(f"API response error - Status code: {response.status_code}, Content: {response.text}")

        data = response.json()
        model_list = []

        edges = data.get("data", {}).get("exploreBotsConnection", {}).get("edges", [])
        for edge in edges:
            handle = edge.get("node", {}).get("handle")
            if handle:
                model_list.append({
                    "id": handle,
                    "object": "model",
                    "created": 0,
                    "owned_by": "poe"
                })

        if not model_list:
            raise PoeError("Model list is empty")

        return {"data": model_list}

# Usage example
client = PoeClient("bot_name", "access_key")
try:
    model_list_response = client.get_model_list(language_code="en")
    print(model_list_response)
except PoeError as e:
    print(e)

{'data': [{'id': 'Assistant', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'Web-Search', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'App-Creator', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'Claude-3.7-Sonnet', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'GPT-4o', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'GPT-4.5-Preview', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'Deepseek-V3-FW', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'Gemini-2.0-Flash', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'Grok-2', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'Claude-3.5-Sonnet', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'Gemini-1.5-Pro', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'Llama-3-70b-Groq', 'object': 'model', 'created': 0, 'owned_by': 'poe'}, {'id': 'Claude-3.5-Haiku', 'object': 'model', 'created': 0, 'owned_by':

通过上述代码，可以获取 poe现在支持的模型，并将其打印出来。
#TODO
将格式化输出的信息在前端展示并通过 openai 的接口调用